In [14]:
import sys

sys.path.append("..")
import numpy as np
from scipy.sparse.linalg import eigsh
import src.fem_1d_eigen
import importlib

importlib.reload(src.fem_1d_eigen)
from src.fem_1d_eigen import *

In [15]:
v = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
femeig = Fem1dEigen(v)
K = femeig.get_K()
M = femeig.get_M()

In [16]:
val, vec = eigsh(K, k=3, which="SM")

In [17]:
val, vec

(array([-6.01804415e-16,  1.20614758e-01,  4.67911114e-01]),
 array([[ 3.33333333e-01, -4.64242827e-01, -4.42975350e-01],
        [ 3.33333333e-01, -4.08248290e-01, -2.35702260e-01],
        [ 3.33333333e-01, -3.03012985e-01,  8.18585360e-02],
        [ 3.33333333e-01, -1.61229842e-01,  3.61116814e-01],
        [ 3.33333333e-01, -1.11022302e-16,  4.71404521e-01],
        [ 3.33333333e-01,  1.61229842e-01,  3.61116814e-01],
        [ 3.33333333e-01,  3.03012985e-01,  8.18585360e-02],
        [ 3.33333333e-01,  4.08248290e-01, -2.35702260e-01],
        [ 3.33333333e-01,  4.64242827e-01, -4.42975350e-01]]))